# Framing plan

In [1]:
# cadquery imports
import math
import numpy
import cadquery as cq
from jupyter_cadquery.cadquery import (Assembly, Part, Edges, Faces, Vertices, show, 
                                       replay, enable_replay, disable_replay, reset_replay)
from jupyter_cadquery import set_sidecar, set_defaults

set_defaults(axes=True, edge_accuracy=0.05, grid=True, axes0=True)

set_sidecar("Framing")
enable_replay()

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


In [21]:
class Wall:
  def __init__(self, width, height_left, height_right):
    self.width = width;
    self.height_left = height_left
    self.height_right = height_right
    self.board_long = 3.5/12
    self.board_short = 1.5/12
    self.spacing = 16/12

  def model(self):
    assembly = []
    angleRad = math.atan((self.height_left - self.height_right) / self.width)
    thicknessFromTopPlateAngle = self.board_short * (1.0/math.cos(angleRad) - 1)
    # baseplate
    assembly.append(cq.Workplane("XY").box(self.width, self.board_long, self.board_short, (False, False, False)))
    # top plate
    top_length = self.width/math.cos(angleRad)
    assembly.append(cq.Workplane("XY").box(top_length, self.board_long, self.board_short, (False, False, False))\
                    .rotate((0,0,0),(0,1,0),math.degrees(angleRad)).translate((0, 0, self.height_left - self.board_short - thicknessFromTopPlateAngle)))
    cutList = {"miter": math.degrees(angleRad), "base plate": self.width, "top plate (along both faces)": top_length}

    studIdx = 1
    # studs
    def stud(x):
      nonlocal studIdx
      height = self.height_left + x * (self.height_right - self.height_left) / self.width
      height = height - 2 * self.board_short - thicknessFromTopPlateAngle - max(0, self.board_short * math.tan(angleRad))
      cutList[f'stud {studIdx} (short side)'] = height
      studIdx = studIdx + 1
      return cq.Workplane("XY").box(self.board_short, self.board_long, height, (False, False, False)).translate((x, 0, self.board_short))
    for x in numpy.arange(0.0, self.width - 2 * self.board_short, self.spacing):
      assembly.append(stud(x))
    assembly.append(stud(self.width - self.board_short))
    print(cutList)

    ## calibration boards
#     assembly.append(cq.Workplane("XY").box(self.width, self.board_long, self.board_short, (False, False, False))\
#                     .translate((0,0,self.height_left-self.board_short)))
#     assembly.append(cq.Workplane("XY").box(self.width, self.board_long, self.board_short, (False, False, False))\
#                     .translate((0,0,self.height_right-self.board_short)))

    return Assembly(list(map(lambda p: Part(p), assembly)))

show(Wall(8, 6, 8).model())

{'miter': -14.036243467926479, 'base plate': 8, 'top plate (along both faces)': 8.246211251235321, 'stud 1 (short side)': 5.746152949199448, 'stud 2 (short side)': 6.079486282532781, 'stud 3 (short side)': 6.412819615866115, 'stud 4 (short side)': 6.746152949199448, 'stud 5 (short side)': 7.079486282532781, 'stud 6 (short side)': 7.412819615866114, 'stud 7 (short side)': 7.714902949199448}
Done, using side car 'Framing'


In [30]:
h_interior = 3.5 # width of 2x4 framing
h_side = 9.0     # from floorplan
h_corner = 4.5   #

v_fence   = 6.0  # height along fence
v_corner = 7.0  # height at hip break
v_peak   = 8.0  # height at the peak

pts =    [ (0,0),   (h_side,0), (h_side,h_corner), (h_corner,h_side), (0,h_side), (0,0)   ]
height = [ v_fence, v_fence,    v_corner,          v_corner,          v_fence,    v_fence ]


walls = []
for i in range(0, len(pts) - 1):
    pt_left = pts[i]
    pt_right = pts[i + 1]
    wall_len = math.sqrt((pt_left[0]-pt_right[0])**2 + (pt_left[1]-pt_right[1])**2)
    walls.append(Wall(wall_len, height[i], height[i+1]).model())

show(Assembly(walls))

{'miter': 0.0, 'base plate': 9.0, 'top plate (along both faces)': 9.0, 'stud 1 (short side)': 5.75, 'stud 2 (short side)': 5.75, 'stud 3 (short side)': 5.75, 'stud 4 (short side)': 5.75, 'stud 5 (short side)': 5.75, 'stud 6 (short side)': 5.75, 'stud 7 (short side)': 5.75, 'stud 8 (short side)': 5.75}
{'miter': -12.528807709151511, 'base plate': 4.5, 'top plate (along both faces)': 4.6097722286464435, 'stud 1 (short side)': 5.746950771426488, 'stud 2 (short side)': 6.043247067722784, 'stud 3 (short side)': 6.339543364019081, 'stud 4 (short side)': 6.635839660315377, 'stud 5 (short side)': 6.71917299364871}
{'miter': 0.0, 'base plate': 6.363961030678928, 'top plate (along both faces)': 6.363961030678928, 'stud 1 (short side)': 6.75, 'stud 2 (short side)': 6.75, 'stud 3 (short side)': 6.75, 'stud 4 (short side)': 6.75, 'stud 5 (short side)': 6.75, 'stud 6 (short side)': 6.75}
{'miter': 12.528807709151511, 'base plate': 4.5, 'top plate (along both faces)': 4.6097722286464435, 'stud 1 (sho